# Los pasos son muy similares:
    
    -filtrar la base de datos
    -hacer todo el rollo de encontrar las oficinas que tienen
    -limpiar un poco la base
    -preparar las cosas de geolocalización
    -graficar el mapa.
    
    **Esta vez comentaré todo menos porque es muy similar a los pasos que seguí en "data_wrangling"

In [1]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
import pymongo
from pandas.io.json import json_normalize

In [2]:
client = pymongo.MongoClient()

In [3]:
db = client.companies_db

In [4]:
data = db.data

In [5]:
competition = db.data.find({'$and': [{'number_of_employees': {'$lte': 100}},\
                                       {'offices':{'$not':{'$size':0}}},\
                                       {'category_code': 'consulting'},
                                      {'founded_year': {'$gte': 2009}}]},
                            {'name':1,'category_code':1,'offices':1, '_id':0})

In [6]:
df = pd.DataFrame(competition)
df.shape

(11, 3)

In [7]:
software = db.data.find({'$and': [{'number_of_employees': {'$lte': 100}},\
                                       {'offices':{'$not':{'$size':0}}},\
                                       {'category_code': 'software'},
                                      {'founded_year': {'$gte': 2009}}]},
                            {'name':1,'category_code':1,'offices':1, '_id':0})

In [8]:
df2 = pd.DataFrame(software)

In [9]:
df3 = pd.concat([df,df2],axis=0)[['name', 'category_code','offices']]

In [10]:
df3.shape

(57, 3)

In [11]:
df3.head(4)

,name,category_code,offices
0,Revenue Architects,consulting,"[{'description': 'Home Office', 'address1': '2..."
1,JumpFox,consulting,"[{'description': 'HQ', 'address1': '544 Natoma..."
2,The Vertical Action Group,consulting,"[{'description': 'World Headquarters', 'addres..."
3,Compel Interaction,consulting,"[{'description': 'Ann Arbor Office', 'address1..."


In [12]:
df_c = df3.dropna(axis=0)

In [13]:
df_c = df_c.reset_index()

In [26]:
#df_c['offices'][0] en realidad no muchas tienen info de la dirección; son 57 empresas en total.

[{'description': 'Home Office',
  'address1': '24 Federal St',
  'address2': '4th Floor',
  'zip_code': '02110',
  'city': 'Boston',
  'state_code': 'MA',
  'country_code': 'USA',
  'latitude': None,
  'longitude': None}]

In [16]:
def get_first(data):
    res=[]
    ofi=[]
    
    data=data['offices']
    for e in data:
        principal=None   # solo las tienen geodata
        if e[0]['latitude'] and e[0]['longitude']:
            principal={'type':'Point',
                       'coordinates':[
                           e[0]['longitude'],
                           e[0]['latitude']
                       ]}
            
        ofi.append(principal)
        
        res.append({
            'total_offices':len(e),
            'lat':e[0]['latitude'],
            'lng':e[0]['longitude'],
            'oficina_principal':principal
        })
    
    return res, ofi

In [17]:
first_office=json_normalize(get_first(df_c)[0])
first_office['oficina_principal']=get_first(df_c)[1]
first_office.head()

,total_offices,lat,lng,oficina_principal,oficina_principal.type,oficina_principal.coordinates
0,1,NaN,NaN,None,NaN,NaN
1,1,NaN,NaN,None,NaN,NaN
2,1,37.798318,-122.400003,"{'type': 'Point', 'coordinates': [-122.4000032...",Point,"[-122.4000032, 37.7983181]"
3,3,42.281569,-83.813914,"{'type': 'Point', 'coordinates': [-83.813914, ...",Point,"[-83.813914, 42.281569]"
4,1,NaN,NaN,None,NaN,NaN


In [22]:
df_c.isnull().sum()

index            0
name             0
category_code    0
offices          0
dtype: int64

In [27]:
df_clean=pd.concat([df_c, first_office], axis=1)[['name', 'category_code', 'lat', 'lng', 'oficina_principal']]

In [30]:
#df_clean.head()

In [29]:
df_clean = df_clean.dropna(axis=0)
df_clean.reset_index(inplace=True)

In [32]:
df_clean.head(3)

,index,name,category_code,lat,lng,oficina_principal
0,2,The Vertical Action Group,consulting,37.798318,-122.400003,"{'type': 'Point', 'coordinates': [-122.4000032..."
1,3,Compel Interaction,consulting,42.281569,-83.813914,"{'type': 'Point', 'coordinates': [-83.813914, ..."
2,5,Fan,consulting,-37.992415,-57.553824,"{'type': 'Point', 'coordinates': [-57.5538238,..."


In [33]:
df4 = pd.DataFrame()
#hice un nuevo dataframe porque por alguna razón en el otro se me hicieron dos índices

In [34]:
df4['name']= df_clean.name
df4['category_code'] = df_clean.category_code
df4['lat'] = df_clean.lat
df4['lng'] = df_clean.lng
df4['oficina_principal'] = df_clean.oficina_principal

In [36]:
df4.head(4)

,name,category_code,lat,lng,oficina_principal
0,The Vertical Action Group,consulting,37.798318,-122.400003,"{'type': 'Point', 'coordinates': [-122.4000032..."
1,Compel Interaction,consulting,42.281569,-83.813914,"{'type': 'Point', 'coordinates': [-83.813914, ..."
2,Fan,consulting,-37.992415,-57.553824,"{'type': 'Point', 'coordinates': [-57.5538238,..."
3,Nalts Consulting,consulting,40.311350,-75.113524,"{'type': 'Point', 'coordinates': [-75.1135236,..."


In [37]:
#db.comptetitors.insert_many(df4.to_dict('records'))
#db.comptetitors.create_index([('oficina_principal', '2dsphere')])
#lo mismo: corrí estos códigos una sola vez.

'oficina_principal_2dsphere'

In [38]:
m = folium.Map(location=[39.8282,-98.5795], tiles='stamentoner', zoom_start=4)

In [39]:
# df4.isnull().sum() ya no hay nulos

name                 0
category_code        0
lat                  0
lng                  0
oficina_principal    0
dtype: int64

In [44]:
list_lat = df4.lat.values
list_long = df4.lng.values
names = df_clean.name.astype(str).values

In [45]:
names

array(['The Vertical Action Group', 'Compel Interaction', 'Fan',
       'Nalts Consulting', 'Outspoken Media', 'MyChances', 'Macroaxis',
       'Carfeine', 'BitRock', 'JumpForward', 'Navara',
       'Unison Technologies', 'Reachoo', 'kinDragon', 'Transifex',
       'AppZero', 'PebbleForge', 'OBE Pro', 'Setster',
       'GameChanger Media', 'crush tweet', 'CineSoft', 'Antelink',
       '1calendar', 'AlphaPoint Technology', "Book'd", 'makandra',
       'DP7 Digital', 'JustProto', 'The Resumator', 'IndiaOnAPage',
       'Nirvana', 'Eventric', 'ResuWe', 'Taxi Fares app', 'Lipperhey',
       'LugIron Software', 'ftopia', 'Tiverias Apps'], dtype=object)